# Algorithm 24: Compute All Atom Coordinates

This algorithm converts backbone frames and torsion angles to full atomic coordinates for all atoms in each residue. It uses ideal bond lengths and angles from protein chemistry.

## Algorithm Pseudocode

![computeAllAtomCoordinates](../imgs/algorithms/computeAllAtomCoordinates.png)

## Source Code Location
- **File**: `AF2-source-code/model/all_atom.py`
- **Functions**: `torsion_angles_to_frames`, `frames_and_literature_positions_to_atom14_pos`
- **Lines**: 400-500

## Overview

```
Input:
├── Backbone frames (R, t) for each residue
├── Torsion angles (ω, φ, ψ, χ1-χ4)
└── Amino acid types
        ↓
Processing:
├── Build rigid group frames using torsion angles
├── Look up literature atom positions for each AA type
└── Transform positions to global coordinates
        ↓
Output:
└── All atom coordinates [N_res, 14, 3] (atom14 format)
```

### Rigid Groups

Each residue has up to 8 rigid groups:
1. Backbone (N, CA, C, O)
2. Pre-omega (previous peptide bond)
3. Phi group
4. Psi group
5-8. Chi1-Chi4 sidechain groups

In [ ]:
import numpy as np

np.random.seed(42)

## NumPy Implementation

In [ ]:
# Standard backbone atom positions (in local frame, Angstroms)
BACKBONE_ATOMS = {
    'N': np.array([0.0, 0.0, 0.0]),
    'CA': np.array([1.458, 0.0, 0.0]),
    'C': np.array([2.009, 1.420, 0.0]),
    'O': np.array([1.251, 2.498, 0.0]),
}

# CB position (for non-glycine)
CB_POSITION = np.array([-0.529, -0.774, 1.205])

# Amino acid names
AA_NAMES = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 
            'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER',
            'THR', 'TRP', 'TYR', 'VAL']

# Glycine index (no CB)
GLY_INDEX = 7

In [ ]:
def make_rotation_matrix(angle, axis='x'):
    """
    Create rotation matrix around specified axis.
    Algorithm 25 for x-axis.
    """
    c, s = np.cos(angle), np.sin(angle)
    
    if axis == 'x':
        R = np.array([[1, 0, 0],
                      [0, c, -s],
                      [0, s, c]])
    elif axis == 'y':
        R = np.array([[c, 0, s],
                      [0, 1, 0],
                      [-s, 0, c]])
    elif axis == 'z':
        R = np.array([[c, -s, 0],
                      [s, c, 0],
                      [0, 0, 1]])
    return R


def make_rotation_matrices_batch(angles, axis='x'):
    """
    Batch version of rotation matrix creation.
    
    Args:
        angles: [N] angles in radians
        axis: 'x', 'y', or 'z'
    
    Returns:
        R: [N, 3, 3] rotation matrices
    """
    N = len(angles)
    c = np.cos(angles)
    s = np.sin(angles)
    
    R = np.zeros((N, 3, 3))
    
    if axis == 'x':
        R[:, 0, 0] = 1
        R[:, 1, 1] = c
        R[:, 1, 2] = -s
        R[:, 2, 1] = s
        R[:, 2, 2] = c
    elif axis == 'y':
        R[:, 0, 0] = c
        R[:, 0, 2] = s
        R[:, 1, 1] = 1
        R[:, 2, 0] = -s
        R[:, 2, 2] = c
    elif axis == 'z':
        R[:, 0, 0] = c
        R[:, 0, 1] = -s
        R[:, 1, 0] = s
        R[:, 1, 1] = c
        R[:, 2, 2] = 1
    
    return R

In [ ]:
def compute_all_atom_coordinates(backbone_frames, torsion_angles, aatype):
    """
    Compute All Atom Coordinates - Algorithm 24.
    
    Converts backbone frames and torsion angles to atom positions.
    
    Args:
        backbone_frames: Tuple (R, t)
            R: [N_res, 3, 3] rotation matrices
            t: [N_res, 3] translations
        torsion_angles: [N_res, 7, 2] sin/cos of torsion angles
            (omega, phi, psi, chi1, chi2, chi3, chi4)
        aatype: [N_res] amino acid types (0-19)
    
    Returns:
        atom14_pos: [N_res, 14, 3] atom positions
    """
    N_res = len(aatype)
    R_bb, t_bb = backbone_frames
    
    print(f"Compute All Atom Coordinates")
    print(f"="*50)
    print(f"Residues: {N_res}")
    print(f"Backbone frames: R {R_bb.shape}, t {t_bb.shape}")
    print(f"Torsion angles: {torsion_angles.shape}")
    
    # Initialize atom positions (14 atoms per residue in atom14 format)
    atom14_pos = np.zeros((N_res, 14, 3))
    
    # Step 1: Place backbone atoms (N, CA, C, O)
    # These are at fixed positions in the local frame
    print(f"\nStep 1: Placing backbone atoms (N, CA, C, O)")
    
    atom_names = ['N', 'CA', 'C', 'O']
    for i, atom_name in enumerate(atom_names):
        local_pos = BACKBONE_ATOMS[atom_name]
        # Transform: global_pos = R @ local_pos + t
        atom14_pos[:, i] = np.einsum('nij,j->ni', R_bb, local_pos) + t_bb
    
    # Step 2: Place CB (for non-glycine residues)
    print(f"Step 2: Placing CB atoms")
    
    is_gly = (aatype == GLY_INDEX)
    cb_pos = np.einsum('nij,j->ni', R_bb, CB_POSITION) + t_bb
    atom14_pos[:, 4] = np.where(is_gly[:, None], 0, cb_pos)  # Index 4 = CB
    
    # Step 3: Build sidechain atoms using chi angles
    print(f"Step 3: Building sidechain atoms using chi angles")
    
    # Convert sin/cos to angles
    chi_angles = np.arctan2(torsion_angles[:, 3:, 0], torsion_angles[:, 3:, 1])
    
    # Simplified sidechain building
    # In reality, this uses residue-specific rigid group definitions
    for chi_idx in range(4):
        atom_idx = 5 + chi_idx  # Atoms 5-8 are sidechain
        if atom_idx >= 14:
            continue
        
        # Build rotation from chi angle
        R_chi = make_rotation_matrices_batch(chi_angles[:, chi_idx], axis='x')
        
        # Place atom along sidechain (simplified)
        bond_length = 1.5  # Angstroms
        local_offset = np.array([bond_length * (chi_idx + 1), 0.0, 0.0])
        
        # Apply chi rotation
        rotated_offset = np.einsum('nij,j->ni', R_chi, local_offset)
        
        # Add to CB position
        atom14_pos[:, atom_idx] = atom14_pos[:, 4] + rotated_offset
    
    print(f"\nOutput: atom14_pos {atom14_pos.shape}")
    
    return atom14_pos

## Test Examples

In [ ]:
# Test 1: Basic functionality
print("Test 1: Basic Functionality")
print("="*60)

N_res = 32

# Initialize backbone frames
R_bb = np.tile(np.eye(3), (N_res, 1, 1))
t_bb = np.zeros((N_res, 3))

# Place residues along a helix-like path
for i in range(N_res):
    t_bb[i, 0] = i * 1.5  # X spacing
    t_bb[i, 1] = np.sin(i * 0.5) * 2  # Y wave
    t_bb[i, 2] = np.cos(i * 0.5) * 2  # Z wave

# Random torsion angles (sin, cos pairs)
angles = np.random.randn(N_res, 7) * 0.5
torsion_angles = np.stack([np.sin(angles), np.cos(angles)], axis=-1)

# Random amino acid types
aatype = np.random.randint(0, 20, size=N_res)

atom14_pos = compute_all_atom_coordinates((R_bb, t_bb), torsion_angles, aatype)

print(f"\nOutput shape: {atom14_pos.shape}")

In [ ]:
# Test 2: Verify backbone geometry
print("\nTest 2: Verify Backbone Geometry")
print("="*60)

# Standard bond lengths (Angstroms)
N_CA_BOND = 1.458
CA_C_BOND = 1.524
C_O_BOND = 1.229

# Compute distances for first residue
N_pos = atom14_pos[0, 0]
CA_pos = atom14_pos[0, 1]
C_pos = atom14_pos[0, 2]
O_pos = atom14_pos[0, 3]

N_CA_dist = np.linalg.norm(CA_pos - N_pos)
CA_C_dist = np.linalg.norm(C_pos - CA_pos)
C_O_dist = np.linalg.norm(O_pos - C_pos)

print(f"N-CA distance: {N_CA_dist:.3f}Å (expected: {N_CA_BOND}Å)")
print(f"CA-C distance: {CA_C_dist:.3f}Å (expected: ~{CA_C_BOND}Å)")
print(f"C-O distance: {C_O_dist:.3f}Å (expected: ~{C_O_BOND}Å)")

In [ ]:
# Test 3: Glycine has no CB
print("\nTest 3: Glycine (no CB)")
print("="*60)

N_res = 10
R_bb = np.tile(np.eye(3), (N_res, 1, 1))
t_bb = np.random.randn(N_res, 3)

angles = np.random.randn(N_res, 7)
torsion_angles = np.stack([np.sin(angles), np.cos(angles)], axis=-1)

# Make some residues glycine
aatype = np.array([0, 7, 1, 7, 2, 7, 3, 7, 4, 5])  # 7 = GLY

atom14_pos = compute_all_atom_coordinates((R_bb, t_bb), torsion_angles, aatype)

# Check CB positions
for i in range(N_res):
    cb_norm = np.linalg.norm(atom14_pos[i, 4])
    is_gly = aatype[i] == 7
    status = "GLY (no CB)" if is_gly else "has CB"
    print(f"Residue {i} ({AA_NAMES[aatype[i]]:3s}): CB norm={cb_norm:.3f} - {status}")

## Verification: Key Properties

In [ ]:
print("Verification: Key Properties")
print("="*60)

np.random.seed(42)
N_res = 24

R_bb = np.tile(np.eye(3), (N_res, 1, 1))
t_bb = np.arange(N_res)[:, None] * np.array([3.8, 0, 0])

angles = np.random.randn(N_res, 7) * 0.3
torsion_angles = np.stack([np.sin(angles), np.cos(angles)], axis=-1)
aatype = np.random.randint(0, 20, size=N_res)

atom14_pos = compute_all_atom_coordinates((R_bb, t_bb), torsion_angles, aatype)

# Property 1: Output shape
shape_correct = atom14_pos.shape == (N_res, 14, 3)
print(f"Property 1 - Shape correct: {shape_correct}")

# Property 2: Finite values
all_finite = np.isfinite(atom14_pos).all()
print(f"Property 2 - All finite: {all_finite}")

# Property 3: Backbone atoms have correct relative positions
n_ca_distances = np.linalg.norm(atom14_pos[:, 1] - atom14_pos[:, 0], axis=-1)
n_ca_correct = np.allclose(n_ca_distances, 1.458, atol=0.01)
print(f"Property 3 - N-CA bond length: {n_ca_correct} (mean={n_ca_distances.mean():.3f}Å)")

# Property 4: Glycine has zero CB
gly_mask = aatype == GLY_INDEX
gly_cb_zero = np.allclose(atom14_pos[gly_mask, 4], 0)
print(f"Property 4 - GLY CB is zero: {gly_cb_zero}")

# Property 5: Non-glycine has non-zero CB
non_gly_mask = aatype != GLY_INDEX
non_gly_cb_nonzero = (np.linalg.norm(atom14_pos[non_gly_mask, 4], axis=-1) > 0).all()
print(f"Property 5 - Non-GLY CB is non-zero: {non_gly_cb_nonzero}")

## Source Code Reference

```python
# From AF2-source-code/model/all_atom.py

def torsion_angles_to_frames(aatype, backb_to_global, torsion_angles_sin_cos):
  """Compute rigid group frames from torsion angles.
  
  Jumper et al. (2021) Suppl. Alg. 24 "computeAllAtomCoordinates"
  
  Builds frames for each rigid group using torsion angles.
  """
  # Get default frames for each residue type
  default_frames = residue_constants.restype_rigid_group_default_frame
  
  # Build rotation matrices from torsion angles
  sin_angles = torsion_angles_sin_cos[..., 0]
  cos_angles = torsion_angles_sin_cos[..., 1]
  
  # Compose with default frames
  ...


def frames_and_literature_positions_to_atom14_pos(aatype, all_frames):
  """Put atoms at literature positions in their frames.
  
  Uses residue_constants.restype_atom14_rigid_group_positions
  to look up atom positions for each residue type.
  """
  # Get literature positions for each residue type
  lit_positions = residue_constants.restype_atom14_rigid_group_positions[aatype]
  
  # Transform to global coordinates
  atom_pos = apply_frames(all_frames, lit_positions)
  
  return atom_pos
```

## Key Insights

1. **Rigid Groups**: Atoms are grouped into rigid bodies that move together. The sidechain is built by chaining these groups.

2. **Literature Positions**: Ideal atom positions are taken from structural chemistry databases, not predicted directly.

3. **Torsion Angles**: Only the torsion angles (dihedral angles) are predicted; bond lengths and angles use ideal values.

4. **Atom14 Format**: A fixed 14-atom representation per residue, with unused positions set to zero.

5. **Glycine Special Case**: Glycine has no CB (beta carbon), so its sidechain slots are zeroed.

6. **Compositional Structure**: The full atom structure is built by composing backbone frame → chi1 frame → chi2 frame → etc.